In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as nc
from gi.repository import NumCosmoMath as ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table

ncm.cfg_init()

cosmo = nc.HICosmo.new_from_name (nc.HICosmo, "NcHICosmoDEXcdm")
primo = nc.HIPrimPowerLaw.new ()

cosmo.add_submodel(primo)

cosmo.props.Omegab  = 0.048254
cosmo.props.Omegac  = 0.30711 - 0.048254
cosmo.props.H0 = 67.7
cosmo.props.Omegax = 1 - 0.30711
primo.props.n_SA = 0.96

dist = nc.Distance.new (2.0)

initialize the `ClusterAbundance` object

In [2]:
#First we need to define the mulltiplicity function here we will use the tinker
mulf = nc.MultiplicityFuncTinker.new()
mulf.set_mdef(nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

#Here we are using the EH transfer function and ml power spectrum with a tophat filter
tf = nc.TransferFunc.new_from_name('NcTransferFuncEH')

psml = nc.PowspecMLTransfer.new(tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = ncm.PowspecFilter.new(psml,ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()
psf.prepare(cosmo)
hmf =  nc.HaloMassFunction.new(dist,psf,mulf)

cluster_m = nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (np.log(1e14),np.log(1e15)))
cluster_z = nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,1.25))

ca = nc.ClusterAbundance.new(hmf,None)
mset = ncm.MSet.new_array([cosmo,cluster_m,cluster_z])
mset.pretty_log()
print('sigma8 = '+str(cosmo.sigma8(psf)))

#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.7                [FIXED]
#   -     Omegac[01]:  0.258856            [FIXED]
#   -     Omegax[02]:  0.69289             [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.048254            [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#----------------------------------------------------------------------------------
# Model parameters
#   - ln10e10ASA[00]:  3.179               [FIXED]
#   - T_SA_ratio[01]:  0.2               

# Binned approach
Compute the 2D integral

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\log_{10}M_\beta}^{\log_{10}M_{\beta + 1}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M
$$

In [9]:
z_nodes = np.linspace(0.25, 1.25, 16)
log10M_nodes = np.linspace(14, 14.8, 11)
lnM_nodes = log10M_nodes * math.log(10)

cluster_m = nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (np.log(1e14),np.log(1e15)))
cluster_z = nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,1.25))

ca.prepare (cosmo, cluster_z, cluster_m)

for zl, zu in zip (z_nodes, z_nodes[1:]):
    for lnMl, lnMu in zip (lnM_nodes, lnM_nodes[1:]):
        Pbin = ca.intp_bin_d2n (cosmo, cluster_z, cluster_m, [lnMl], [lnMu], None, [zl], [zu], None)
        print ("(% 20.13g % 20.13g) (% 20.13g % 20.13g) % 20.13g" % (lnMl / math.log(10), lnMu / math.log(10), zl, zu, Pbin))


(                  14                14.08) (                0.25      0.3166666666667)       52.33457713229
(               14.08                14.16) (                0.25      0.3166666666667)        40.8232888887
(               14.16                14.24) (                0.25      0.3166666666667)       31.49337137628
(               14.24                14.32) (                0.25      0.3166666666667)       23.99344552052
(               14.32                 14.4) (                0.25      0.3166666666667)       18.02240762824
(                14.4                14.48) (                0.25      0.3166666666667)       13.32187864299
(               14.48                14.56) (                0.25      0.3166666666667)       9.669921957072
(               14.56                14.64) (                0.25      0.3166666666667)       6.875862118753
(               14.64                14.72) (                0.25      0.3166666666667)       4.776057324548
(               14.

# Un-binned approach

Compute $$\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M_i,z_i)}{d\log_{10}M_i}$$ for each masses and redshifts in a dark matter halo catalog

In [ ]:
ncdata = nc.DataClusterNCount.new(ca)
rng = ncm.RNG.pool_get("example_ca_sampling");
ncdata.init_from_sampling(mset,(0.25)*4*np.pi,rng)

ncdata.catalog_save("ca_nodist_binned_numcosmo.fits", True)
ncdata_fits = fits.open('ca_nodist_binned_numcosmo.fits')
#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort('Z_TRUE')
d2n = []
for i in ncdata_Table:
    d2n.append(ca.d2n(cosmo,cluster_z,cluster_m,i[1],i[0]))
    
d2n


In [ ]:
import sys
sys.path.insert(0,'/home/henrique/Programas_Cosmologia/CLCosmo_Sim/cluster_abundance/')
import pyccl as ccl
cosmo_ccl = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8884898175540845, n_s=0.96, Omega_k = 0)
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count

clc = cl_count.ClusterAbundance()
#choose the halo mass function and mass definition
massdef = ccl.halos.massdef.MassDef(200, 'critical', c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncTinker08(cosmo_ccl, mass_def=massdef)
clc.set_cosmology(cosmo = cosmo_ccl, hmd = hmd, massdef = massdef)
clc.sky_area = (0.25)*4*np.pi

log10M = np.array(ncdata_Table[1][:]*np.log10(np.exp(1)))
multiplicity_exact = clc.multiplicity_function_individual_MZ(z = np.array(ncdata_Table[0][:]), logm =log10M, method = 'exact')
multiplicity_exact


In [ ]:
z_grid = np.linspace(0., 3, 500)
logm_grid = np.linspace(12,16, 500)
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
multiplicity_interp = clc.multiplicity_function_individual_MZ(z = np.array(ncdata_Table[0][:]), logm =log10M, method = 'interp')
multiplicity_interp